In [10]:
import pandas as pd
import torch

In [2]:
!pip3 install sentence-transformers


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [11]:
from sentence_transformers import SentenceTransformer, util

In [12]:
# load responses
responses_df = pd.read_csv('./GPT responses.csv')
responses_df

,Full Text,Relevant Phrase,Predicate,ID subgroups,Annotator,ChatGPT response,Response outcome,prompt,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Jews are late,are late,are late,NaN,Jon,are punctual,opposite,"From the following sentence, extract a stateme...",Ashkenazi,NaN,Conservative,NaN,NaN
1,"""I don't know much about Jews except that you'...",are good with money,are good with money,NaN,Jon,are good with money,correct,"From the following sentence, extract a stateme...",Ashkenazi,NaN,I would describe myself as...,"Reform-thinking, Conservative-practicing",NaN
2,Jews expect special treatment as a minority gr...,expect special treatment,expect special treatment,NaN,Jon,expect special treatment,correct,"From the following sentence, extract a stateme...",Prefer not to say,NaN,Just Jewish,NaN,NaN
3,"Dont go to church, whats it like-beung Jewish/...",don't go to church,don't go to church,NaN,Jon,kill Jesus,correct,"From the following sentence, extract a stateme...",Ashkenazi,NaN,"Reconstructionist,Renewal,Havurah,Culturally J...",NaN,NaN
4,"Dont go to church, whats it like-beung Jewish/...",different,are different,NaN,Jon,worship differently,correct,"From the following sentence, extract a stateme...",Ashkenazi,NaN,"Reconstructionist,Renewal,Havurah,Culturally J...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,*I have been called an oppressor \n*I have bee...,called a colonizer,are colonizers,NaN,Jon,are powerful,hallucination,"From the following sentence, extract a stateme...",Ashkenazi,NaN,Reform,NaN,NaN
207,*I have been called an oppressor \n*I have bee...,called a racist,are racist,NaN,Jon,are manipulative,hallucination,"From the following sentence, extract a stateme...",Ashkenazi,NaN,Reform,NaN,NaN
208,Anything surrounding Israel\nJewish people are...,are greedy,are greedy,NaN,Jon,are greedy,correct,"From the following sentence, extract a stateme...",Ashkenazi,NaN,"Conservative,Just Jewish,Culturally Jewish",NaN,NaN
209,people made fun of me because I was the smart ...,was the smart kid,are smart,NaN,Jon,All Jews were blamed.,hallucination,"From the following sentence, extract a stateme...",Ashkenazi,NaN,Conservative,NaN,NaN


In [13]:
results_df = pd.DataFrame(columns=['Full Text', 'Predicate', 'ID subgroups', 'Best response', 'Exact Match', 'Phrase Cosine Sim.', 'Sent. Cosine Sim.', 'Extraction Outcome'])


In [14]:
embed_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [15]:
# NB: efficiency could be improved here with groupby and a lambda function, but the data is small enough it shouldn't matter too much

# find closest predicate for each response 
for index, row in responses_df.iterrows():
    # lowercase, strip periods and extra whitespace
    if row.isnull()['Predicate']:
        print("null pred")
        continue
    pred = row['Predicate'].lower().replace(".", '').strip()
    # get all correct predicates for this sentence
    responses = responses_df[responses_df['Full Text'] == row['Full Text']]['ChatGPT response'].unique().tolist()
    # lowercase, strip periods and extra whitespace
    for r in responses:
        r = r.lower().replace(".", '').strip()
    
    # ChatGPT response that best matches this predicate
    best_response = ''
    exact = 0
    # check exact matches
    if pred in responses:
        exact = 1
    
    # calculate SBERT cosine similarity 
    pred_embed = embed_model.encode([pred])
    resp_embed = embed_model.encode(responses)
    
    cosine_scores = util.cos_sim(pred_embed, resp_embed)
    # get max cosine score and its index
    best_index = torch.argmax(cosine_scores)
    best_response = responses[best_index]
    best_score = cosine_scores[0][best_index].item()
    
    # calculate SBERT cosine similarity for dummy sentence
    sent_pred = "All Jewish people are " + pred
    sent_responses = ["All Jewish people are " + r for r in responses]
    sp_embed = embed_model.encode([sent_pred])
    sr_embed = embed_model.encode(sent_responses)
    
    s_cosine_scores = util.cos_sim(sp_embed, sr_embed)
    # get max cosine score and its index
    s_best_index = torch.argmax(cosine_scores)
    if s_best_index != best_index:
        print("difference in best match!")
    s_best_response = responses[s_best_index]
    s_best_score = s_cosine_scores[0][s_best_index].item()
    
    # write row to results_df
    results_df.at[index, 'Full Text'] = row['Full Text']
    results_df.at[index, 'Predicate'] = row['Predicate']
    results_df.at[index, 'ID subgroups'] = row['ID subgroups'] if row.notnull()['ID subgroups'] else "Jewish"
    results_df.at[index, 'Best response'] = best_response
    results_df.at[index, 'Exact Match'] = exact
    results_df.at[index, 'Phrase Cosine Sim.'] = best_score
    results_df.at[index, 'Extraction Outcome'] = row['Response outcome']
    results_df.at[index, 'Sent. Cosine Sim.'] = s_best_score
    
    print("wrote row", index)

wrote row 0
wrote row 1
wrote row 2
wrote row 3
wrote row 4
wrote row 5
wrote row 6
wrote row 7
wrote row 8
wrote row 9
wrote row 10
wrote row 11
wrote row 12
wrote row 13
wrote row 14
wrote row 15
wrote row 16
wrote row 17
wrote row 18
wrote row 19
wrote row 20
wrote row 21
wrote row 22
wrote row 23
wrote row 24
wrote row 25
wrote row 26
wrote row 27
wrote row 28
wrote row 29
wrote row 30
wrote row 31
wrote row 32
wrote row 33
wrote row 34
wrote row 35
wrote row 36
wrote row 37
wrote row 38
wrote row 39
wrote row 40
wrote row 41
wrote row 42
wrote row 43
wrote row 44
wrote row 45
wrote row 46
wrote row 47
wrote row 48
wrote row 49
wrote row 50
wrote row 51
wrote row 52
wrote row 53
wrote row 54
wrote row 55
wrote row 56
wrote row 57
wrote row 58
wrote row 59
wrote row 60
wrote row 61
wrote row 62
wrote row 63
wrote row 64
wrote row 65
wrote row 66
wrote row 67
wrote row 68
wrote row 69
wrote row 70
wrote row 71
wrote row 72
wrote row 73
wrote row 74
wrote row 75
wrote row 76
wrote row

In [8]:
results_df

,Full Text,Predicate,ID subgroups,Best response,Exact Match,Phrase Cosine Sim.,Sent. Cosine Sim.,Extraction Outcome
0,Jews are late,are late,Jewish,are punctual,0,0.618577,0.828589,opposite
1,"""I don't know much about Jews except that you'...",are good with money,Jewish,are good with money,1,1.0,1.0,correct
2,Jews expect special treatment as a minority gr...,expect special treatment,Jewish,expect special treatment,1,1.0,1.0,correct
3,"Dont go to church, whats it like-beung Jewish/...",don't go to church,Jewish,kill Jesus,0,0.446777,0.662429,correct
4,"Dont go to church, whats it like-beung Jewish/...",are different,Jewish,believe in different things,0,0.345289,0.813228,correct
...,...,...,...,...,...,...,...,...
206,*I have been called an oppressor \n*I have bee...,are colonizers,Jewish,are greedy,0,0.275132,0.636283,hallucination
207,*I have been called an oppressor \n*I have bee...,are racist,Jewish,are greedy,0,0.502888,0.676046,hallucination
208,Anything surrounding Israel\nJewish people are...,are greedy,Jewish,are greedy,1,1.0,1.0,correct
209,people made fun of me because I was the smart ...,are smart,Jewish,All Jews were blamed.,0,0.104006,0.551251,hallucination


In [22]:
results_df['Phrase Cosine Sim.'].mean()

0.6092528160661459

In [23]:
results_df['Sent. Cosine Sim.'].mean()

0.8175807016737321

In [24]:
results_df['Exact Match'].value_counts()

Exact Match
0    167
1     37
Name: count, dtype: int64

In [25]:
results_df['Extraction Outcome'].value_counts(normalize=True)

Extraction Outcome
correct          0.455882
grammar          0.196078
hallucination    0.196078
other            0.117647
opposite         0.034314
Name: proportion, dtype: float64

In [19]:
text = results_df[(results_df['Extraction Outcome'] == 'other') & (results_df['ID subgroups'] == 'Jewish')]['Full Text']

In [21]:
for t in text:
    print(t)

 By anti-Semites  scrubbed and beaten up and called bad names;   other discriminated against us for not  seeking middle class Jewish box and rejecting us from community and services
I cannot recall ever experiencing an antisemitic slur directed at me particularly. I have had antisemitic slurs directed at groups I was part of by onlookers (i.e. during March of the Living having someone yell at our group). I have also experienced people making generalized slurs against the Jewish people in front of me not realizing I was Jewish. Lastly, I have experienced antisemitic graffiti (swastikas) in public places that were not directed at me but still felt harmful.
Yes. That we lie about needing extra/different days off to observe Jewish holidays
Jewish people are more loyal to Israel than their home countries.
Jews love money: When I was a child teenagers on the top deck of a London bus threw small change down at me and my father. It was very confusing and I didn't understand why I was told to k